In [ ]:
import pandas as pd

file_path = r"C:\Users\charl\OneDrive\Documents\GitHub\5-CSRTT\data_performance\2025_12_12__17_50_08_659.csv"

# Skip the first 9 rows (metadata header), allow variable number of fields, handle empty cells
df = pd.read_csv(
    file_path,
    skiprows=9,
    engine='python',
    on_bad_lines='warn',
    keep_default_na=False,
    na_values=[''],
    header=0
)

# Rename columns
column_names = ["Num_line", "S", "MS", "Cat", "Num_cat", "State", "Display", "null"]
df.columns = column_names[:len(df.columns)]

df

In [ ]:
# Identify trials: each trial starts with "MagEntry" where Cat is "Entry" and ends with the next "MagEntry" with Cat "Entry"
# Find all indices where "MagEntry" appears in State column AND "Entry" in Cat column
magentry_indices = df[(df['State'] == 'MagEntry') & (df['Cat'] == 'Entry')].index.tolist()

# Create a list to store trial information
trials = []

# For each pair of consecutive MagEntry Entry events
for i in range(len(magentry_indices) - 1):
    trial_start_idx = magentry_indices[i]
    trial_end_idx = magentry_indices[i + 1]
    
    # Extract the trial data (from start to end, inclusive of end)
    trial_data = df.loc[trial_start_idx:trial_end_idx].copy()
    
    # Get all states/categories in this trial
    trial_states = trial_data['State'].tolist()
    
    # Store trial information using actual dataframe indices
    trials.append({
        'trial_number': i + 1,
        'start_line': trial_start_idx,
        'end_line': trial_end_idx,
        'start_idx': trial_start_idx,
        'end_idx': trial_end_idx,
        'states': trial_states,
        'num_events': len(trial_data)
    })

# Display trials summary
for trial in trials:
    print(f"Trial {trial['trial_number']}: Rows {trial['start_line']} to {trial['end_line']} ({trial['num_events']} events)")
    print(f"  States: {' -> '.join(trial['states'])}")
    print()

print(f"Total trials identified: {len(trials)}")

In [ ]:
# Load df_659_12_12.csv and create detailed trials summary with timestamps
import pandas as pd
import json

file_path = r"C:\Users\charl\OneDrive\Documents\GitHub\5-CSRTT\df_659_12_12.csv"

# Load the data - NO skiprows since first row is header only
df = pd.read_csv(
    file_path,
    engine='python',
    on_bad_lines='warn',
    keep_default_na=False,
    na_values=[''],
    header=0
)

# Rename columns
column_names = ["Num_line", "S", "MS", "Cat", "Num_cat", "State", "Display", "null"]
df.columns = column_names[:len(df.columns)]

# Display the dataframe with actual row numbers to verify
print("Data loaded. Showing first 22 rows:")
for idx in range(min(22, len(df))):
    actual_row = idx + 2  # Convert pandas index to actual row number (row 1 = header)
    print(f"Row {actual_row}: State={df.loc[idx, 'State']}, Cat={df.loc[idx, 'Cat']}")
print(f"\nDataframe has {len(df)} rows of data\n")

# Identify trials: MagEntry with Entry
magentry_indices = df[(df['State'] == 'MagEntry') & (df['Cat'] == 'Entry')].index.tolist()
print(f"Found MagEntry+Entry at pandas indices: {magentry_indices}")
print(f"Found MagEntry+Entry at actual CSV rows: {[idx + 2 for idx in magentry_indices]}\n")

# Create detailed trials list with state information and timestamps
detailed_trials = []

for i in range(len(magentry_indices) - 1):
    trial_start_idx = magentry_indices[i]
    trial_end_idx = magentry_indices[i + 1]
    
    # Convert to actual row numbers (add 2: 1 for header row + 1 for 1-based indexing)
    actual_start_row = trial_start_idx + 2
    actual_end_row = trial_end_idx + 2
    
    # Extract trial data
    trial_data = df.loc[trial_start_idx:trial_end_idx].copy()
    
    # Create list of states with timestamps
    states_with_time = []
    for idx, row in trial_data.iterrows():
        actual_row_num = idx + 2
        states_with_time.append({
            'actual_row': actual_row_num,
            'state': row['State'],
            'cat': row['Cat'],
            'seconds': row['S'],
            'milliseconds': row['MS'],
            'display': row['Display']
        })
    
    # Store trial information with ACTUAL row numbers
    detailed_trials.append({
        'trial_number': i + 1,
        'start_row': actual_start_row,
        'end_row': actual_end_row,
        'num_events': len(trial_data),
        'states_sequence': ' -> '.join([s['state'] for s in states_with_time]),
        'states_detailed': json.dumps(states_with_time)
    })

# Create DataFrame and save to CSV
trials_df = pd.DataFrame(detailed_trials)

output_path = r"C:\Users\charl\OneDrive\Documents\GitHub\5-CSRTT\trials_summary.csv"
trials_df.to_csv(output_path, index=False)

print(f"Detailed trials summary saved to: {output_path}")
print(f"Total trials identified: {len(detailed_trials)}\n")

# Display all trials with ACTUAL row numbers
for trial in detailed_trials:
    print(f"Trial {trial['trial_number']}: Rows {trial['start_row']} to {trial['end_row']} ({trial['num_events']} events)")
    print(f"  States: {trial['states_sequence']}")
    print()

trials_df

In [ ]:
import pandas as pd

# Read the CSV files
df_data = pd.read_csv('df_659_12_12.csv')
trials_summary = pd.read_csv('trials_summary.csv')

# Function to format time as "S.MS"
def format_time(s, ms):
    return f"{s}.{ms}"

# Process each trial
for idx, row in trials_summary.iterrows():
    trial_num = int(row['trial_number'])
    start_row = int(row['start_row'])
    end_row = int(row['end_row'])
    
    # Get the slice of data for this trial (adjusting for 0-based indexing)
    # start_row and end_row are 1-indexed actual CSV rows (with row 1 being header)
    # So pandas index = actual_row - 2
    # We exclude end_row since that's the start of the next trial
    trial_data = df_data.iloc[start_row-2:end_row-2]
    
    # Find MagEntry timestamp (Entry state) - should only be one now (the start)
    mag_entry = trial_data[(trial_data['Cat'] == 'Entry') & (trial_data['State'] == 'MagEntry')]
    if not mag_entry.empty:
        mag_s = mag_entry.iloc[0]['S']
        mag_ms = mag_entry.iloc[0]['MS']
        trials_summary.at[idx, 'MagEntry'] = format_time(mag_s, mag_ms)
    
    # Find On1A1 timestamp (Input state)
    on1a1 = trial_data[(trial_data['Cat'] == 'Input') & (trial_data['State'] == 'On1A1')]
    if not on1a1.empty:
        on1a1_s = on1a1.iloc[0]['S']
        on1a1_ms = on1a1.iloc[0]['MS']
        trials_summary.at[idx, 'On1A1'] = format_time(on1a1_s, on1a1_ms)
    
    print(f"Processed trial {trial_num}: MagEntry={trials_summary.at[idx, 'MagEntry']}, On1A1={trials_summary.at[idx, 'On1A1']}")

# Save the updated trials_summary
trials_summary.to_csv('trials_summary.csv', index=False)
print("\nDone! Updated trials_summary.csv")


In [ ]:
import pandas as pd

# Read the trials_summary CSV
trials_summary = pd.read_csv('trials_summary.csv')

# Function to convert "S.MS" format to total seconds as float
def time_to_seconds(time_str):
    if pd.isna(time_str):
        return None
    parts = str(time_str).split('.')
    if len(parts) == 2:
        seconds = float(parts[0])
        milliseconds = float(parts[1])
        return seconds + (milliseconds / 1000.0)
    return float(time_str)

# Function to convert seconds back to "S.MS" format
def seconds_to_time(seconds):
    if seconds is None or pd.isna(seconds):
        return None
    s = int(seconds)
    ms = int((seconds - s) * 1000)
    return f"{s}.{ms}"

# Calculate Delta for each row
delta_values = []
for idx, row in trials_summary.iterrows():
    mag_entry_time = time_to_seconds(row['MagEntry'])
    on1a1_time = time_to_seconds(row['On1A1'])
    
    if mag_entry_time is not None and on1a1_time is not None:
        delta_seconds = on1a1_time - mag_entry_time
        delta_formatted = seconds_to_time(delta_seconds)
        delta_values.append(delta_formatted)
        print(f"Trial {row['trial_number']}: On1A1({on1a1_time:.3f}s) - MagEntry({mag_entry_time:.3f}s) = Delta({delta_formatted})")
    else:
        delta_values.append(None)
        print(f"Trial {row['trial_number']}: Missing timestamp data")

# Insert Delta column right after On1A1
col_position = trials_summary.columns.get_loc('On1A1') + 1
trials_summary.insert(col_position, 'Delta', delta_values)

# Save the updated CSV
trials_summary.to_csv('trials_summary.csv', index=False)
print("\nDone! Added Delta column to trials_summary.csv")

trials_summary[['trial_number', 'MagEntry', 'On1A1', 'Delta']].head(10)

In [ ]:
import pandas as pd

# Read the CSV files
df_data = pd.read_csv('df_659_12_12.csv')
trials_summary = pd.read_csv('trials_summary.csv')

# Function to format time as "S.MS"
def format_time(s, ms):
    return f"{s}.{ms}"

# Process each trial to find ITI_start timestamp
iti_start_values = []
for idx, row in trials_summary.iterrows():
    trial_num = int(row['trial_number'])
    start_row = int(row['start_row'])
    end_row = int(row['end_row'])
    
    # Get the slice of data for this trial (adjusting for 0-based indexing)
    trial_data = df_data.iloc[start_row-2:end_row-2]
    
    # Find ITI2sec timestamp (Entry state)
    iti_entry = trial_data[(trial_data['Cat'] == 'Entry') & (trial_data['State'] == 'ITI2sec')]
    if not iti_entry.empty:
        iti_s = iti_entry.iloc[0]['S']
        iti_ms = iti_entry.iloc[0]['MS']
        iti_timestamp = format_time(iti_s, iti_ms)
        iti_start_values.append(iti_timestamp)
        print(f"Trial {trial_num}: ITI_start = {iti_timestamp}")
    else:
        iti_start_values.append(None)
        print(f"Trial {trial_num}: No ITI2sec Entry found")

# Add ITI_start column to trials_summary
trials_summary['ITI_start'] = iti_start_values

# Save the updated CSV
trials_summary.to_csv('trials_summary.csv', index=False)
print("\nDone! Added ITI_start column to trials_summary.csv")

trials_summary[['trial_number', 'MagEntry', 'On1A1', 'Delta', 'ITI_start']].head(10)

In [ ]:
import pandas as pd

# Read the CSV files
df_data = pd.read_csv('df_659_12_12.csv')
trials_summary = pd.read_csv('trials_summary.csv')

# Function to format time as "S.MS"
def format_time(s, ms):
    return f"{s}.{ms}"

# Process each trial to find T_Premat timestamp
t_premat_values = []
for idx, row in trials_summary.iterrows():
    trial_num = int(row['trial_number'])
    start_row = int(row['start_row'])
    end_row = int(row['end_row'])
    
    # Get the slice of data for this trial (adjusting for 0-based indexing)
    trial_data = df_data.iloc[start_row-2:end_row-2]
    
    # Find Premature timestamp (Entry state)
    premature_entry = trial_data[(trial_data['Cat'] == 'Entry') & (trial_data['State'] == 'Premature')]
    if not premature_entry.empty:
        prem_s = premature_entry.iloc[0]['S']
        prem_ms = premature_entry.iloc[0]['MS']
        prem_timestamp = format_time(prem_s, prem_ms)
        t_premat_values.append(prem_timestamp)
        print(f"Trial {trial_num}: T_Premat = {prem_timestamp}")
    else:
        t_premat_values.append('0')
        print(f"Trial {trial_num}: No Premature Entry found - set to 0")

# Add T_Premat column to trials_summary
trials_summary['T_Premat'] = t_premat_values

# Save the updated CSV
trials_summary.to_csv('trials_summary.csv', index=False)
print("\nDone! Added T_Premat column to trials_summary.csv")

trials_summary[['trial_number', 'MagEntry', 'On1A1', 'ITI_start', 'T_Premat']].head(10)

In [ ]:
import pandas as pd

# Read the trials_summary CSV
trials_summary = pd.read_csv('trials_summary.csv')

# Function to convert "S.MS" format to total seconds as float
def time_to_seconds(time_str):
    if pd.isna(time_str) or str(time_str) == '0':
        return None
    parts = str(time_str).split('.')
    if len(parts) == 2:
        seconds = float(parts[0])
        milliseconds = float(parts[1])
        return seconds + (milliseconds / 1000.0)
    return float(time_str)

# Function to convert seconds back to "S.MS" format
def seconds_to_time(seconds):
    if seconds is None or pd.isna(seconds):
        return None
    s = int(seconds)
    ms = int((seconds - s) * 1000)
    return f"{s}.{ms}"

# Calculate ITI_premat_delta for each row
iti_premat_delta_values = []
for idx, row in trials_summary.iterrows():
    t_premat = row['T_Premat']
    
    # Check if T_Premat is '0' or 0 or invalid (handle both string and numeric)
    if pd.isna(t_premat) or str(t_premat) == '0' or t_premat == 0:
        iti_premat_delta_values.append('NaN')
        print(f"Trial {row['trial_number']}: No premature response - ITI_premat_delta = NaN")
    else:
        # Convert timestamps to seconds
        t_premat_seconds = time_to_seconds(t_premat)
        iti_start_seconds = time_to_seconds(row['ITI_start'])
        
        if t_premat_seconds is not None and iti_start_seconds is not None:
            # Calculate difference: T_Premat - ITI_start
            delta_seconds = t_premat_seconds - iti_start_seconds
            delta_formatted = seconds_to_time(delta_seconds)
            iti_premat_delta_values.append(delta_formatted)
            print(f"Trial {row['trial_number']}: T_Premat({t_premat_seconds:.3f}s) - ITI_start({iti_start_seconds:.3f}s) = {delta_formatted}")
        else:
            iti_premat_delta_values.append('NaN')
            print(f"Trial {row['trial_number']}: Missing timestamp data - ITI_premat_delta = NaN")

# Add ITI_premat_delta column to trials_summary
trials_summary['ITI_premat_delta'] = iti_premat_delta_values

# Try to save, if file is locked, show warning but continue
try:
    trials_summary.to_csv('trials_summary.csv', index=False)
    print("\nDone! Added ITI_premat_delta column to trials_summary.csv")
except PermissionError:
    print("\nWarning: Could not save to trials_summary.csv (file may be open). Close the file and run this cell again to save.")

trials_summary[['trial_number', 'ITI_start', 'T_Premat', 'ITI_premat_delta']].head(10)

In [ ]:
import pandas as pd
import numpy as np

# Read the CSV files
df_data = pd.read_csv('df_659_12_12.csv')
trials_summary = pd.read_csv('trials_summary.csv')

# Create a list to store CorrectResp values
correctresp_values = []

# Process each trial
for idx, row in trials_summary.iterrows():
    trial_num = int(row['trial_number'])
    start_row = int(row['start_row'])
    end_row = int(row['end_row'])
    
    # Get the slice of data for this trial
    # start_row and end_row are 1-indexed actual CSV rows (with row 1 being header)
    # So we need to adjust: pandas index = actual_row - 2 (1 for header, 1 for 0-indexing)
    trial_data = df_data.iloc[start_row-2:end_row-2]
    
    # Find the first occurrence of CorrectResp in State column where Cat is 'Entry'
    correctresp_entry = trial_data[(trial_data['State'] == 'CorrectResp') & (trial_data['Cat'] == 'Entry')]
    
    if not correctresp_entry.empty:
        # If CorrectResp with Entry is found, add 'CorrectResp' to the list
        correctresp_values.append('CorrectResp')
        print(f"Trial {trial_num}: CorrectResp found")
    else:
        # If not found, add NaN
        correctresp_values.append(np.nan)
        print(f"Trial {trial_num}: No CorrectResp")

# Add CorrectResp column to trials_summary
trials_summary['CorrectResp'] = correctresp_values

# Try to save, if file is locked, show warning but continue
try:
    trials_summary.to_csv('trials_summary.csv', index=False)
    print("\nDone! Added CorrectResp column to trials_summary.csv")
except PermissionError:
    print("\nWarning: Could not save to trials_summary.csv (file may be open). Close the file and run this cell again to save.")

# Display a sample of the results
trials_summary[['trial_number', 'start_row', 'end_row', 'CorrectResp']].head(20)

In [ ]:
import pandas as pd
import numpy as np

# Read the CSV files
df_data = pd.read_csv('df_659_12_12.csv')
trials_summary = pd.read_csv('trials_summary.csv')

# Create a list to store IncorrectResp values
incorrectresp_values = []

# Process each trial
for idx, row in trials_summary.iterrows():
    trial_num = int(row['trial_number'])
    start_row = int(row['start_row'])
    end_row = int(row['end_row'])
    
    # Get the slice of data for this trial
    # start_row and end_row are 1-indexed actual CSV rows (with row 1 being header)
    # So we need to adjust: pandas index = actual_row - 2 (1 for header, 1 for 0-indexing)
    trial_data = df_data.iloc[start_row-2:end_row-2]
    
    # Find the first occurrence of IncorrectResp in State column where Cat is 'Entry'
    incorrectresp_entry = trial_data[(trial_data['State'] == 'IncorrectResp') & (trial_data['Cat'] == 'Entry')]
    
    if not incorrectresp_entry.empty:
        # If IncorrectResp with Entry is found, add 'IncorrectResp' to the list
        incorrectresp_values.append('IncorrectResp')
        print(f"Trial {trial_num}: IncorrectResp found")
    else:
        # If not found, add NaN
        incorrectresp_values.append(np.nan)
        print(f"Trial {trial_num}: No IncorrectResp")

# Add IncorrectResp column to trials_summary
trials_summary['IncorrectResp'] = incorrectresp_values

# Try to save, if file is locked, show warning but continue
try:
    trials_summary.to_csv('trials_summary.csv', index=False)
    print("\nDone! Added IncorrectResp column to trials_summary.csv")
except PermissionError:
    print("\nWarning: Could not save to trials_summary.csv (file may be open). Close the file and run this cell again to save.")

# Display a sample of the results
trials_summary[['trial_number', 'start_row', 'end_row', 'IncorrectResp']].head(20)

In [ ]:
import pandas as pd
import numpy as np

# Read the CSV files
df_data = pd.read_csv('df_659_12_12.csv')
trials_summary = pd.read_csv('trials_summary.csv')

# Create a list to store Omission values
omission_values = []

# Process each trial
for idx, row in trials_summary.iterrows():
    trial_num = int(row['trial_number'])
    start_row = int(row['start_row'])
    end_row = int(row['end_row'])
    
    # Get the slice of data for this trial
    # start_row and end_row are 1-indexed actual CSV rows (with row 1 being header)
    # So we need to adjust: pandas index = actual_row - 2 (1 for header, 1 for 0-indexing)
    trial_data = df_data.iloc[start_row-2:end_row-2]
    
    # Find the first occurrence of Omission in State column where Cat is 'Entry'
    omission_entry = trial_data[(trial_data['State'] == 'Omission') & (trial_data['Cat'] == 'Entry')]
    
    if not omission_entry.empty:
        # If Omission with Entry is found, add 'Omission' to the list
        omission_values.append('Omission')
        print(f"Trial {trial_num}: Omission found")
    else:
        # If not found, add NaN
        omission_values.append(np.nan)
        print(f"Trial {trial_num}: No Omission")

# Add Omission column to trials_summary
trials_summary['Omission'] = omission_values

# Try to save, if file is locked, show warning but continue
try:
    trials_summary.to_csv('trials_summary.csv', index=False)
    print("\nDone! Added Omission column to trials_summary.csv")
except PermissionError:
    print("\nWarning: Could not save to trials_summary.csv (file may be open). Close the file and run this cell again to save.")

# Display a sample of the results
trials_summary[['trial_number', 'start_row', 'end_row', 'Omission']].head(20)

In [ ]:
import pandas as pd
import numpy as np

# Read the CSV files
df_data = pd.read_csv('df_659_12_12.csv')
trials_summary = pd.read_csv('trials_summary.csv')

# Function to format time as "S.MS"
def format_time(s, ms):
    return f"{s}.{ms}"

# Create a list to store cue_on values
cue_on_values = []

# Process each trial
for idx, row in trials_summary.iterrows():
    trial_num = int(row['trial_number'])
    start_row = int(row['start_row'])
    end_row = int(row['end_row'])
    
    # Get the slice of data for this trial
    # start_row and end_row are 1-indexed actual CSV rows (with row 1 being header)
    # So we need to adjust: pandas index = actual_row - 2 (1 for header, 1 for 0-indexing)
    trial_data = df_data.iloc[start_row-2:end_row-2]
    
    # Find the first occurrence of holeX (where X is 1-5) in State column where Cat is 'Entry'
    # Check for any state that matches "hole1", "hole2", "hole3", "hole4", or "hole5"
    hole_entry = trial_data[
        (trial_data['State'].str.match(r'^hole[1-5]$', na=False)) & 
        (trial_data['Cat'] == 'Entry')
    ]
    
    if not hole_entry.empty:
        # Get the timestamp from the first occurrence
        hole_s = hole_entry.iloc[0]['S']
        hole_ms = hole_entry.iloc[0]['MS']
        cue_on_timestamp = format_time(hole_s, hole_ms)
        cue_on_values.append(cue_on_timestamp)
        hole_state = hole_entry.iloc[0]['State']
        print(f"Trial {trial_num}: cue_on = {cue_on_timestamp} (found {hole_state})")
    else:
        # If not found, add NaN
        cue_on_values.append(np.nan)
        print(f"Trial {trial_num}: No hole Entry found")

# Add cue_on column to trials_summary
trials_summary['cue_on'] = cue_on_values

# Try to save, if file is locked, show warning but continue
try:
    trials_summary.to_csv('trials_summary.csv', index=False)
    print("\nDone! Added cue_on column to trials_summary.csv")
except PermissionError:
    print("\nWarning: Could not save to trials_summary.csv (file may be open). Close the file and run this cell again to save.")

# Display a sample of the results
trials_summary[['trial_number', 'start_row', 'end_row', 'CorrectResp', 'cue_on']].head(20)

In [ ]:
import pandas as pd
import numpy as np

# Read the CSV files
df_data = pd.read_csv('df_659_12_12.csv')
trials_summary = pd.read_csv('trials_summary.csv')

# Function to format time as "S.MS"
def format_time(s, ms):
    return f"{s}.{ms}"

# Create a list to store t_correct values
t_correct_values = []

# Process each trial
for idx, row in trials_summary.iterrows():
    trial_num = int(row['trial_number'])
    start_row = int(row['start_row'])
    end_row = int(row['end_row'])
    
    # Get the slice of data for this trial
    # start_row and end_row are 1-indexed actual CSV rows (with row 1 being header)
    # So we need to adjust: pandas index = actual_row - 2 (1 for header, 1 for 0-indexing)
    trial_data = df_data.iloc[start_row-2:end_row-2]
    
    # Check if this trial has CorrectResp
    if pd.notna(row.get('CorrectResp')) and row.get('CorrectResp') == 'CorrectResp':
        # Find the first occurrence of CorrectResp in State column where Cat is 'Entry'
        correctresp_entry = trial_data[(trial_data['State'] == 'CorrectResp') & (trial_data['Cat'] == 'Entry')]
        
        if not correctresp_entry.empty:
            # Get the timestamp from the first occurrence
            correct_s = correctresp_entry.iloc[0]['S']
            correct_ms = correctresp_entry.iloc[0]['MS']
            t_correct_timestamp = format_time(correct_s, correct_ms)
            t_correct_values.append(t_correct_timestamp)
            print(f"Trial {trial_num}: t_correct = {t_correct_timestamp}")
        else:
            # If not found (shouldn't happen), add NaN
            t_correct_values.append(np.nan)
            print(f"Trial {trial_num}: CorrectResp marked but timestamp not found")
    else:
        # If trial doesn't have CorrectResp, add NaN
        t_correct_values.append(np.nan)
        print(f"Trial {trial_num}: No CorrectResp")

# Add t_correct column to trials_summary
trials_summary['t_correct'] = t_correct_values

# Try to save, if file is locked, show warning but continue
try:
    trials_summary.to_csv('trials_summary.csv', index=False)
    print("\nDone! Added t_correct column to trials_summary.csv")
except PermissionError:
    print("\nWarning: Could not save to trials_summary.csv (file may be open). Close the file and run this cell again to save.")

# Display a sample of the results
trials_summary[['trial_number', 'CorrectResp', 'cue_on', 't_correct']].head(20)

In [ ]:
import pandas as pd
import numpy as np

# Read the trials_summary CSV
trials_summary = pd.read_csv('trials_summary.csv')

# Function to convert "S.MS" format to total seconds as float
def time_to_seconds(time_str):
    if pd.isna(time_str):
        return None
    parts = str(time_str).split('.')
    if len(parts) == 2:
        seconds = float(parts[0])
        milliseconds = float(parts[1])
        return seconds + (milliseconds / 1000.0)
    return float(time_str)

# Function to convert seconds back to "S.MS" format
def seconds_to_time(seconds):
    if seconds is None or pd.isna(seconds):
        return None
    s = int(seconds)
    ms = int((seconds - s) * 1000)
    return f"{s}.{ms}"

# Calculate latency_cue_onset for each row
latency_cue_onset_values = []
for idx, row in trials_summary.iterrows():
    trial_num = row['trial_number']
    cue_on_time = time_to_seconds(row.get('cue_on'))
    t_correct_time = time_to_seconds(row.get('t_correct'))
    
    if cue_on_time is not None and t_correct_time is not None:
        # Calculate difference: t_correct - cue_on
        latency_seconds = t_correct_time - cue_on_time
        latency_formatted = seconds_to_time(latency_seconds)
        latency_cue_onset_values.append(latency_formatted)
        print(f"Trial {trial_num}: t_correct({t_correct_time:.3f}s) - cue_on({cue_on_time:.3f}s) = latency_cue_onset({latency_formatted})")
    else:
        latency_cue_onset_values.append(np.nan)
        print(f"Trial {trial_num}: Missing timestamp data for latency calculation")

# Add latency_cue_onset column to trials_summary
trials_summary['latency_cue_onset'] = latency_cue_onset_values

# Try to save, if file is locked, show warning but continue
try:
    trials_summary.to_csv('trials_summary.csv', index=False)
    print("\nDone! Added latency_cue_onset column to trials_summary.csv")
except PermissionError:
    print("\nWarning: Could not save to trials_summary.csv (file may be open). Close the file and run this cell again to save.")

# Display a sample of the results
trials_summary[['trial_number', 'CorrectResp', 'cue_on', 't_correct', 'latency_cue_onset']].head(20)

In [ ]:
import pandas as pd
import numpy as np

# Read the CSV files
df_data = pd.read_csv('df_659_12_12.csv')
trials_summary = pd.read_csv('trials_summary.csv')

# Function to format time as "S.MS"
def format_time(s, ms):
    return f"{s}.{ms}"

# Create a list to store T_IncorrectResp values
t_incorrectresp_values = []

# Process each trial
for idx, row in trials_summary.iterrows():
    trial_num = int(row['trial_number'])
    start_row = int(row['start_row'])
    end_row = int(row['end_row'])
    
    # Get the slice of data for this trial
    # start_row and end_row are 1-indexed actual CSV rows (with row 1 being header)
    # So we need to adjust: pandas index = actual_row - 2 (1 for header, 1 for 0-indexing)
    trial_data = df_data.iloc[start_row-2:end_row-2]
    
    # Check if this trial has IncorrectResp
    if pd.notna(row.get('IncorrectResp')) and row.get('IncorrectResp') == 'IncorrectResp':
        # Find the first occurrence of IncorrectResp in State column where Cat is 'Entry'
        incorrectresp_entry = trial_data[(trial_data['State'] == 'IncorrectResp') & (trial_data['Cat'] == 'Entry')]
        
        if not incorrectresp_entry.empty:
            # Get the timestamp from the first occurrence
            incorrect_s = incorrectresp_entry.iloc[0]['S']
            incorrect_ms = incorrectresp_entry.iloc[0]['MS']
            t_incorrectresp_timestamp = format_time(incorrect_s, incorrect_ms)
            t_incorrectresp_values.append(t_incorrectresp_timestamp)
            print(f"Trial {trial_num}: T_IncorrectResp = {t_incorrectresp_timestamp}")
        else:
            # If not found (shouldn't happen), add NaN
            t_incorrectresp_values.append(np.nan)
            print(f"Trial {trial_num}: IncorrectResp marked but timestamp not found")
    else:
        # If trial doesn't have IncorrectResp, add NaN
        t_incorrectresp_values.append(np.nan)
        print(f"Trial {trial_num}: No IncorrectResp")

# Add T_IncorrectResp column to trials_summary
trials_summary['T_IncorrectResp'] = t_incorrectresp_values

# Try to save, if file is locked, show warning but continue
try:
    trials_summary.to_csv('trials_summary.csv', index=False)
    print("\nDone! Added T_IncorrectResp column to trials_summary.csv")
except PermissionError:
    print("\nWarning: Could not save to trials_summary.csv (file may be open). Close the file and run this cell again to save.")

# Display a sample of the results
trials_summary[['trial_number', 'IncorrectResp', 'cue_on', 'T_IncorrectResp']].head(20)

In [4]:
import pandas as pd
import numpy as np

# Read the trials_summary CSV
trials_summary = pd.read_csv('trials_summary.csv')

# Function to convert "S.MS" format to total seconds as float
def time_to_seconds(time_str):
    if pd.isna(time_str):
        return None
    parts = str(time_str).split('.')
    if len(parts) == 2:
        seconds = float(parts[0])
        # Pad milliseconds string with zeros on the right to make it 3 digits
        # "7" becomes "700", "65" becomes "650", "635" stays "635"
        ms_str = parts[1].ljust(3, '0')
        milliseconds = float(ms_str)
        return seconds + (milliseconds / 1000.0)
    return float(time_str)

# Function to convert seconds back to "S.MS" format
def seconds_to_time(seconds):
    if seconds is None or pd.isna(seconds):
        return None
    s = int(seconds)
    ms = int(round((seconds - s) * 1000))
    # Format milliseconds with 3 digits, padding with zeros
    return f"{s}.{ms:03d}"

# Calculate latency_incorrect_cue_onset for each row
latency_incorrect_cue_onset_values = []
for idx, row in trials_summary.iterrows():
    trial_num = row['trial_number']
    cue_on_time = time_to_seconds(row.get('cue_on'))
    t_incorrectresp_time = time_to_seconds(row.get('T_IncorrectResp'))
    
    if cue_on_time is not None and t_incorrectresp_time is not None:
        # Calculate difference: T_IncorrectResp - cue_on
        latency_seconds = t_incorrectresp_time - cue_on_time
        latency_formatted = seconds_to_time(latency_seconds)
        latency_incorrect_cue_onset_values.append(latency_formatted)
        print(f"Trial {trial_num}: T_IncorrectResp({t_incorrectresp_time:.3f}s) - cue_on({cue_on_time:.3f}s) = latency_incorrect_cue_onset({latency_formatted})")
    else:
        latency_incorrect_cue_onset_values.append(np.nan)
        print(f"Trial {trial_num}: Missing timestamp data for incorrect latency calculation")

# Add latency_incorrect_cue_onset column to trials_summary
trials_summary['latency_incorrect_cue_onset'] = latency_incorrect_cue_onset_values

# Try to save, if file is locked, show warning but continue
try:
    trials_summary.to_csv('trials_summary.csv', index=False)
    print("\nDone! Added latency_incorrect_cue_onset column to trials_summary.csv")
except PermissionError:
    print("\nWarning: Could not save to trials_summary.csv (file may be open). Close the file and run this cell again to save.")

# Display a sample of the results
trials_summary[['trial_number', 'IncorrectResp', 'cue_on', 'T_IncorrectResp', 'latency_incorrect_cue_onset']].head(20)

Trial 1: Missing timestamp data for incorrect latency calculation
Trial 2: Missing timestamp data for incorrect latency calculation
Trial 3: Missing timestamp data for incorrect latency calculation
Trial 4: Missing timestamp data for incorrect latency calculation
Trial 5: Missing timestamp data for incorrect latency calculation
Trial 6: Missing timestamp data for incorrect latency calculation
Trial 7: Missing timestamp data for incorrect latency calculation
Trial 8: Missing timestamp data for incorrect latency calculation
Trial 9: T_IncorrectResp(189.700s) - cue_on(189.635s) = latency_incorrect_cue_onset(0.065)
Trial 10: Missing timestamp data for incorrect latency calculation
Trial 11: Missing timestamp data for incorrect latency calculation
Trial 12: T_IncorrectResp(227.555s) - cue_on(227.500s) = latency_incorrect_cue_onset(0.055)
Trial 13: Missing timestamp data for incorrect latency calculation
Trial 14: Missing timestamp data for incorrect latency calculation
Trial 15: Missing tim

,trial_number,IncorrectResp,cue_on,T_IncorrectResp,latency_incorrect_cue_onset
0,1,NaN,5.590,NaN,NaN
1,2,NaN,65.730,NaN,NaN
2,3,NaN,83.795,NaN,NaN
3,4,NaN,88.320,NaN,NaN
4,5,NaN,NaN,NaN,NaN
5,6,NaN,NaN,NaN,NaN
6,7,NaN,150.460,NaN,NaN
7,8,NaN,NaN,NaN,NaN
8,9,IncorrectResp,189.635,189.700,0.065
9,10,NaN,NaN,NaN,NaN


In [5]:
import pandas as pd
import numpy as np

# Read the trials_summary CSV
trials_summary = pd.read_csv('trials_summary.csv')

# Count the number of CorrectResp and IncorrectResp
num_correct = trials_summary['CorrectResp'].value_counts().get('CorrectResp', 0)
num_incorrect = trials_summary['IncorrectResp'].value_counts().get('IncorrectResp', 0)

# Calculate accuracy percentage
total_responses = num_correct + num_incorrect
if total_responses > 0:
    accuracy_percent = (num_correct / total_responses) * 100
else:
    accuracy_percent = 0

print(f"Number of Correct Responses: {num_correct}")
print(f"Number of Incorrect Responses: {num_incorrect}")
print(f"Total Responses: {total_responses}")
print(f"Accuracy: {accuracy_percent:.2f}%")

# Create Accuracy column with the percentage only in the first row
accuracy_values = [accuracy_percent] + [np.nan] * (len(trials_summary) - 1)
trials_summary['Accuracy'] = accuracy_values

# Try to save, if file is locked, show warning but continue
try:
    trials_summary.to_csv('trials_summary.csv', index=False)
    print(f"\nDone! Added Accuracy column to trials_summary.csv with {accuracy_percent:.2f}% in the first row")
except PermissionError:
    print("\nWarning: Could not save to trials_summary.csv (file may be open). Close the file and run this cell again to save.")

# Display a sample of the results
trials_summary[['trial_number', 'CorrectResp', 'IncorrectResp', 'Accuracy']].head(10)

Number of Correct Responses: 213
Number of Incorrect Responses: 8
Total Responses: 221
Accuracy: 96.38%

Done! Added Accuracy column to trials_summary.csv with 96.38% in the first row


,trial_number,CorrectResp,IncorrectResp,Accuracy
0,1,NaN,NaN,96.38009
1,2,NaN,NaN,NaN
2,3,CorrectResp,NaN,NaN
3,4,NaN,NaN,NaN
4,5,NaN,NaN,NaN
5,6,NaN,NaN,NaN
6,7,NaN,NaN,NaN
7,8,NaN,NaN,NaN
8,9,NaN,IncorrectResp,NaN
9,10,NaN,NaN,NaN


In [6]:
import pandas as pd
import numpy as np

# Read the trials_summary CSV
trials_summary = pd.read_csv('trials_summary.csv')

# Count the number of CorrectResp, IncorrectResp, and Omission
num_correct = trials_summary['CorrectResp'].value_counts().get('CorrectResp', 0)
num_incorrect = trials_summary['IncorrectResp'].value_counts().get('IncorrectResp', 0)
num_omission = trials_summary['Omission'].value_counts().get('Omission', 0)

# Calculate accuracy percentage including omissions
total_responses_with_omission = num_correct + num_incorrect + num_omission
if total_responses_with_omission > 0:
    accuracy_with_omission_percent = (num_correct / total_responses_with_omission) * 100
else:
    accuracy_with_omission_percent = 0

print(f"Number of Correct Responses: {num_correct}")
print(f"Number of Incorrect Responses: {num_incorrect}")
print(f"Number of Omissions: {num_omission}")
print(f"Total Responses (including omissions): {total_responses_with_omission}")
print(f"Accuracy+Om: {accuracy_with_omission_percent:.2f}%")

# Create Accuracy+Om column with the percentage only in the first row
accuracy_om_values = [accuracy_with_omission_percent] + [np.nan] * (len(trials_summary) - 1)
trials_summary['Accuracy+Om'] = accuracy_om_values

# Try to save, if file is locked, show warning but continue
try:
    trials_summary.to_csv('trials_summary.csv', index=False)
    print(f"\nDone! Added Accuracy+Om column to trials_summary.csv with {accuracy_with_omission_percent:.2f}% in the first row")
except PermissionError:
    print("\nWarning: Could not save to trials_summary.csv (file may be open). Close the file and run this cell again to save.")

# Display a sample of the results
trials_summary[['trial_number', 'CorrectResp', 'IncorrectResp', 'Omission', 'Accuracy', 'Accuracy+Om']].head(10)

Number of Correct Responses: 213
Number of Incorrect Responses: 8
Number of Omissions: 62
Total Responses (including omissions): 283
Accuracy+Om: 75.27%

Done! Added Accuracy+Om column to trials_summary.csv with 75.27% in the first row


,trial_number,CorrectResp,IncorrectResp,Omission,Accuracy,Accuracy+Om
0,1,NaN,NaN,Omission,96.38009,75.265018
1,2,NaN,NaN,Omission,NaN,NaN
2,3,CorrectResp,NaN,NaN,NaN,NaN
3,4,NaN,NaN,Omission,NaN,NaN
4,5,NaN,NaN,NaN,NaN,NaN
5,6,NaN,NaN,NaN,NaN,NaN
6,7,NaN,NaN,Omission,NaN,NaN
7,8,NaN,NaN,NaN,NaN,NaN
8,9,NaN,IncorrectResp,NaN,NaN,NaN
9,10,NaN,NaN,NaN,NaN,NaN


In [7]:
import pandas as pd
import numpy as np

# Read the trials_summary CSV
trials_summary = pd.read_csv('trials_summary.csv')

# Count the number of Omissions
num_omission = trials_summary['Omission'].value_counts().get('Omission', 0)

# Get total number of trials
total_trials = len(trials_summary)

# Calculate omission percentage
if total_trials > 0:
    omission_percent = (num_omission / total_trials) * 100
else:
    omission_percent = 0

print(f"Number of Omissions: {num_omission}")
print(f"Total Trials: {total_trials}")
print(f"Omission %: {omission_percent:.2f}%")

# Create Omission % column with the percentage only in the first row
omission_percent_values = [omission_percent] + [np.nan] * (len(trials_summary) - 1)
trials_summary['Omission %'] = omission_percent_values

# Try to save, if file is locked, show warning but continue
try:
    trials_summary.to_csv('trials_summary.csv', index=False)
    print(f"\nDone! Added Omission % column to trials_summary.csv with {omission_percent:.2f}% in the first row")
except PermissionError:
    print("\nWarning: Could not save to trials_summary.csv (file may be open). Close the file and run this cell again to save.")

# Display a sample of the results
trials_summary[['trial_number', 'Omission', 'Accuracy', 'Accuracy+Om', 'Omission %']].head(10)

Number of Omissions: 62
Total Trials: 292
Omission %: 21.23%

Done! Added Omission % column to trials_summary.csv with 21.23% in the first row


,trial_number,Omission,Accuracy,Accuracy+Om,Omission %
0,1,Omission,96.38009,75.265018,21.232877
1,2,Omission,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN
3,4,Omission,NaN,NaN,NaN
4,5,NaN,NaN,NaN,NaN
5,6,NaN,NaN,NaN,NaN
6,7,Omission,NaN,NaN,NaN
7,8,NaN,NaN,NaN,NaN
8,9,NaN,NaN,NaN,NaN
9,10,NaN,NaN,NaN,NaN


In [ ]:
import pandas as pd
import numpy as np

# Read the trials_summary CSV
trials_summary = pd.read_csv('trials_summary.csv')

# Create binary series: 1 for Omission, 0 for non-omission
binary_series = trials_summary['Omission'].apply(lambda x: 1 if x == 'Omission' else 0)

print("Binary series (1 = Omission, 0 = No Omission):")
print(binary_series.values)
print()

# Find sequences of consecutive omissions
sequences = []
current_sequence_length = 0

for value in binary_series:
    if value == 1:
        # Omission found, increment current sequence
        current_sequence_length += 1
    else:
        # No omission, end current sequence if there was one
        if current_sequence_length > 0:
            sequences.append(current_sequence_length)
            current_sequence_length = 0

# Don't forget to add the last sequence if it ends with omissions
if current_sequence_length > 0:
    sequences.append(current_sequence_length)

print(f"Omission sequences found: {sequences}")
print(f"Number of sequences: {len(sequences)}")

# Calculate the average length of omission sequences
if len(sequences) > 0:
    omission_seq_avg = sum(sequences) / len(sequences)
else:
    omission_seq_avg = 0

print(f"Average omissions per sequence: {omission_seq_avg:.2f}")

# Create Omission_seq_avg column with the average only in the first row
omission_seq_values = [omission_seq_avg] + [np.nan] * (len(trials_summary) - 1)
trials_summary['Omission_seq_avg'] = omission_seq_values

# Try to save, if file is locked, show warning but continue
try:
    trials_summary.to_csv('trials_summary.csv', index=False)
    print(f"\nDone! Added Omission_seq_avg column to trials_summary.csv with {omission_seq_avg:.2f} in the first row")
except PermissionError:
    print("\nWarning: Could not save to trials_summary.csv (file may be open). Close the file and run this cell again to save.")

# Display a sample of the results
trials_summary[['trial_number', 'Omission', 'Omission %', 'Omission_seq_avg']].head(10)

Binary series (1 = Omission, 0 = No Omission):
[1 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0
 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 1 0 1 0 0 0 0 0 1
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 1 1 0 1 0 1 0
 1 0 0 0 0 0 1 0 0 0 0 0 1 1 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0
 0 0 1 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 1 1 0 0 0 1 0 0 0 0 1]

Omission sequences found: [2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 3, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 3, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1]
Number of sequences: 49
Average omissions per sequence: 1.27

Done! Added Omission_seq column to trials_summary.csv with 1.27 in the first row


,trial_number,Omission,Omission %,Omission_seq
0,1,Omission,21.232877,1.265306
1,2,Omission,NaN,NaN
2,3,NaN,NaN,NaN
3,4,Omission,NaN,NaN
4,5,NaN,NaN,NaN
5,6,NaN,NaN,NaN
6,7,Omission,NaN,NaN
7,8,NaN,NaN,NaN
8,9,NaN,NaN,NaN
9,10,NaN,NaN,NaN


In [9]:
import pandas as pd
import numpy as np

# Read the trials_summary CSV
trials_summary = pd.read_csv('trials_summary.csv')

# Create binary series: 1 for Omission, 0 for non-omission
binary_series = trials_summary['Omission'].apply(lambda x: 1 if x == 'Omission' else 0)

# Find sequences of consecutive omissions
sequences = []
current_sequence_length = 0

for value in binary_series:
    if value == 1:
        # Omission found, increment current sequence
        current_sequence_length += 1
    else:
        # No omission, end current sequence if there was one
        if current_sequence_length > 0:
            sequences.append(current_sequence_length)
            current_sequence_length = 0

# Don't forget to add the last sequence if it ends with omissions
if current_sequence_length > 0:
    sequences.append(current_sequence_length)

print(f"Omission sequences found: {sequences}")

# Calculate max sequence length and total number of sequences
if len(sequences) > 0:
    max_seq_om = max(sequences)
    amount_seq_om = len(sequences)
else:
    max_seq_om = 0
    amount_seq_om = 0

print(f"Maximum sequence length (max_seq_om): {max_seq_om}")
print(f"Total number of sequences (amount_seq_om): {amount_seq_om}")

# Create max_seq_om column with the value only in the first row
max_seq_om_values = [max_seq_om] + [np.nan] * (len(trials_summary) - 1)
trials_summary['max_seq_om'] = max_seq_om_values

# Create amount_seq_om column with the value only in the first row
amount_seq_om_values = [amount_seq_om] + [np.nan] * (len(trials_summary) - 1)
trials_summary['amount_seq_om'] = amount_seq_om_values

# Try to save, if file is locked, show warning but continue
try:
    trials_summary.to_csv('trials_summary.csv', index=False)
    print(f"\nDone! Added max_seq_om ({max_seq_om}) and amount_seq_om ({amount_seq_om}) columns to trials_summary.csv")
except PermissionError:
    print("\nWarning: Could not save to trials_summary.csv (file may be open). Close the file and run this cell again to save.")

# Display a sample of the results
trials_summary[['trial_number', 'Omission', 'Omission_seq', 'max_seq_om', 'amount_seq_om']].head(10)

Omission sequences found: [2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 3, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 3, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1]
Maximum sequence length (max_seq_om): 3
Total number of sequences (amount_seq_om): 49

Done! Added max_seq_om (3) and amount_seq_om (49) columns to trials_summary.csv


,trial_number,Omission,Omission_seq,max_seq_om,amount_seq_om
0,1,Omission,1.265306,3.0,49.0
1,2,Omission,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN
3,4,Omission,NaN,NaN,NaN
4,5,NaN,NaN,NaN,NaN
5,6,NaN,NaN,NaN,NaN
6,7,Omission,NaN,NaN,NaN
7,8,NaN,NaN,NaN,NaN
8,9,NaN,NaN,NaN,NaN
9,10,NaN,NaN,NaN,NaN
